In [41]:
import pandas as pd

In [1]:
sc

<SparkContext master=local[4] appName=PySparkShell>

In [2]:
spark

In [32]:
cln_list = []

with open('data/avl/sfmtaAVLRawData01032016.csv') as infile:
    for line in infile:
        line = line.strip()
        
        if len(line) > 125:
            cln_list.append(line[0:89])
            cln_list.append(line[89:])
        
        if len(line) != 0 and len(line) < 125:
            cln_list.append(line)

with open('data/avl/clean_sample.csv', 'w') as outfile:
    for row in cln_list:
        outfile.write(row + '\n')
            

yolo 149


In [36]:
# read CSV
df = spark.read.csv('data/avl/clean_sample.csv',
                         header=True,       # use headers or not
                         quote='"',         # char for quotes
                         sep=",",           # char for separation
                         inferSchema=True)  # do we infer schema or not ?


In [37]:
# prints the schema
df.printSchema()

# some functions are still valid
print("line count: {}".format(df.count()))

# show the table in a oh-so-nice format
df.show()

root
 |-- REV: integer (nullable = true)
 |-- REPORT_TIME: string (nullable = true)
 |-- VEHICLE_TAG: string (nullable = true)
 |-- LONGITUDE: double (nullable = true)
 |-- LATITUDE: double (nullable = true)
 |-- SPEED: double (nullable = true)
 |-- HEADING: double (nullable = true)
 |-- TRAIN_ASSIGNMENT: string (nullable = true)
 |-- PREDICTABLE: integer (nullable = true)

line count: 494778
+----+-------------------+-----------+----------+--------+-----+-------+----------------+-----------+
| REV|        REPORT_TIME|VEHICLE_TAG| LONGITUDE|LATITUDE|SPEED|HEADING|TRAIN_ASSIGNMENT|PREDICTABLE|
+----+-------------------+-----------+----------+--------+-----+-------+----------------+-----------+
|1506|01/03/2016 00:03:29|         01|-122.42047|37.80633|  0.0|  201.0|            null|          0|
|1506|01/03/2016 00:04:59|         01|-122.42049|37.80638|  0.0|  256.0|            null|          0|
|1506|01/03/2016 00:12:29|         01|-122.42092| 37.8069|  0.0|  247.0|            null|     

In [39]:
df.describe('SPEED').show()

+-------+-----------------+
|summary|            SPEED|
+-------+-----------------+
|  count|           494778|
|   mean|4.265489779254438|
| stddev|4.961628821231181|
|    min|              0.0|
|    max|           67.778|
+-------+-----------------+



Let's see if we can throw in distance...

In [47]:
from pyspark.sql.functions import udf
from geopy import distance
from pyspark.sql.types import FloatType


def distance_function(lat,lon):
    
    start_latlon = (37.786908, -122.45656)
    row_latlon = (float(lat), float(lon))
    
    return distance.distance(start_latlon, row_latlon).m


distance_udf = udf(lambda lat, lon: distance_function(lat, lon), FloatType())

df_out = df.withColumn("distance", distance_udf(df['LATITUDE'], df['LONGITUDE']))

# df_out.select('Date','High', 'Low', 'Open', 'Close', 'special').show()
df_out.select('distance').show()

+---------+
| distance|
+---------+
|3840.6492|
|3842.3093|
|3844.0718|
| 3844.791|
| 3846.151|
| 3832.928|
|3840.9644|
|3836.4036|
|3804.4297|
|3716.1511|
|3689.3696|
| 3607.872|
| 3553.213|
|3524.5383|
|3494.3171|
|3486.0679|
|3555.6938|
| 3630.813|
| 3708.431|
|3940.2405|
+---------+
only showing top 20 rows

